<a href="https://colab.research.google.com/github/RK22000/FOLd/blob/main/FOLd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This document is divided into 3 sections:

1. Helper Methods

        Has all the helper methods and their minor tests. (Skipable) (Please skip it, or rather only check it if you know which helper method you are looking for)
1. Main Code
    
        Has the code for the main brains of the First Order Logic.
1. Tests
    
        Shows the FOL engine at work with test cases. Can be checked before reading other sections.



My reccommended way to read this document is from the bottom up. First start from the earliest test cases and read up to the higher test cases. By the time you reach the top you will have understood what the FOL_engine is. Then when you look into the FOL_engine in section 2, you will have an idea of what is being done and why. 

# Section 1: Helper methods

In [44]:
import re

In [45]:
re.findall(r'\(.*\)', "Smart(x)(y)")

['(x)(y)']

In [46]:
re.findall(r'\([^(]*\)', "Smart(x)(y)")

['(x)', '(y)']

In [47]:
#match = re.findall(r'\((?P<arguments>[^()]*)\)', "Smart(x, y) Clever(p, (q))")
b1 = r'\(([^)(]*)\)'
b2 = r'\(([^)(]*' + r'\([^)(]\)' + r'[^()]*)\)'

match = re.findall(b1+'|'+b2, "Smart(x, y) Clever(p, (q))")
match#.group("arguments")

[('x, y', ''), ('', 'p, (q)')]

In [48]:
import re
def getFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate.
    Basically a text parser so """
    regex = re.compile(r'(?P<negation>~?)(?P<function>\w+)\((?P<arguments>[^()]*)\)')
    match = regex.search(predicate)
    arguments = match.group("arguments").split(',')
    for i, argument in enumerate(arguments):
        arguments[i] = argument.strip()
    func = match.group("function")
    negation = match.group("negation")
    return func, arguments, negation


In [49]:
getFuncArgs("~Smart(x, y)")

('Smart', ['x', 'y'], '~')

In [50]:
import re
def isFuncArgs(predicate: str):
    """Get the first function and its set of arguments in the predicate"""
    regex = re.compile(r'^(?P<function>~?\w+)\((?P<arguments>[^()]*)\)$')
    match = regex.search(predicate)
    return True if match else False


In [51]:
isFuncArgs("Smart(x)^Clever(x)")

False

In [52]:
isFuncArgs("abc")

False

In [53]:
def validTerm(term: str):
    regex = re.compile(r"^[A-Za-z]+$")
    return True if regex.search(term) else False

def groundTerm(term: str):
    if not validTerm(term=term): return False
    regex = re.compile(r"^[a-z]+$")
    return True if regex.search(term) else False

def constantTerm(term: str):
    if not validTerm(term=term): return False
    regex = re.compile(r"^[A-Z][A-Za-z]+$")
    return True if regex.search(term) else False

In [54]:
groundTerm('x'), constantTerm('RahulKan'), validTerm('XyZ')

(True, True, True)

In [55]:
# Issue: this does not see a problem in this case [x, x], [A, B]. x cant be A and B
# For now the issue hasn't surfaced because I hadn't created a test case for this.
def swapable(f_terms: list, p_terms: list):
    count=0
    if not len(f_terms) == len(p_terms): return False
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term) or f_term==p_term:
            count+=1
    return count==len(f_terms)

In [56]:
None == True, None == False

(False, False)

In [57]:
def isRule(sentence: str):
    return sentence.count("=>") == 1

In [58]:
isRule(" a => b")

True

In [59]:
def getCondImpl(sentence: str):
    condition, implication = sentence.split("=>")
    return condition.strip(), implication.strip()

In [60]:
getCondImpl(" a ^ x=> b ")

('a ^ x', 'b')

In [61]:
def swap_list(f_terms: list, p_terms: list): 
    swap = dict()
    if not len(f_terms) == len(p_terms): return swap
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term): 
            swap[f_term] = p_term
    return swap

In [62]:
def replace_var(var: str, rpl: str, sentence:str) -> str:
    def swap(match: re.Match):
        #print(f'Match {match}')
        return match.group()[0] + rpl + match.group()[-1]
    wrapper = r"[\s)(,]"
    pattern = wrapper + f"(?P<var>{var})" + wrapper
    return re.sub(pattern=pattern, repl=swap, string=sentence)

In [63]:
replace_var("y", "Azula", "Prodigy(y)")

'Prodigy(Azula)'

In [64]:
def conjunction(sentence: str) -> bool:
    conjuncts = set( [conj.strip() for conj in sentence.split("^")] )
    return "^" in sentence and '' not in conjuncts

In [65]:
conjunction(" a ^ B  ^ ^ x")

False

In [66]:
def getConjuncts(sentence: str) -> set:
    return set([conj.strip() for conj in sentence.split("^")])


In [67]:
getConjuncts(" a ^ B ^ x ")

{'B', 'a', 'x'}

In [68]:
def unifyable(f_terms: list, p_terms: list):
    count=0
    if not len(f_terms) == len(p_terms): return False
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term) or groundTerm(p_term) or f_term==p_term:
            count+=1
    return count==len(f_terms)

In [69]:
unifyable(["Aang"], ["x"]), unifyable(["g"], ["Aang"])

(True, True)

In [70]:
def unified_arguments(f_terms: list, p_terms: list):
    swap = dict()
    if not len(f_terms) == len(p_terms): return swap
    for f_term, p_term in zip(f_terms, p_terms):
        if groundTerm(f_term):
            swap[f_term] = p_term
        elif groundTerm(p_term):
            swap[p_term] = f_term
    return swap

# Section 2: Main Code

In [72]:
class Fol_engine:
    def __init__(self, facts: set = set(), rules: set = set()) -> None:
        self.facts = facts
        self.rules = rules

    def trueCase(self, predicate: str):
        """ This method uses getTrueCases to make an inference.
        """

        # If the predicate is a conjunction, then split in and check if the individual parts are true
        if conjunction(sentence=predicate):
            conjuncts = getConjuncts(sentence=predicate)
            print(f"Conjunct of {conjuncts}")
            result = True
            for conjunct in conjuncts:
                val = self.trueCase(conjunct)
                if val == False: # val may be False or None
                    return False
                elif val is None:
                    result = None
            return result

        else:
            cases = self.getCases(predicate=predicate)
            if predicate in cases[True]: return True
            elif predicate in cases[False]: return False
            else: return None
        


    def getCases(self, predicate: str):
        """ This method has the ability to guess.
        It takes in a predicate and returns a dictionary mapped to 2 sets
        
        _key__|_set___________________________________________________________
         True | A sets of true predicates where the ground terms in the original 
        ______|_predicates_have_been_substituted_with_some_constants__________
        False | A sets of false predicates where the ground terms in the original 
        ______|_predicates_have_been_substituted_with_some_constants__________

        You can see this in test case 8.0 and 8.0.2

        This method also works if there are no ground terms. In which case it 
        just puts the predicate in either the true list or the false list

        You can see this in 8.0.1

        I'm not sure how to deal with conjunctions over here. I think I sould be 
        able to solve the Co. West problem (Test case 8.1) if I figure out how to 
        deal with conjunctions.
        """
        if conjunction(sentence=predicate):
            # I can split the conjunction and get cases for each conjunct, but I dont know what to do next
            pass

        # The predicate is like ~Abc(x, y)
        if isFuncArgs(predicate):
            p_func, p_arguments, p_negation = getFuncArgs(predicate) # Abc, [x, y], ~
            print(f"Getting cases for {predicate}")
            true_cases = set()
            false_cases = set()
            cases = {True: true_cases, False: false_cases}

            # First look to see if we know a usefull fact
            for fact in self.facts:
                if isFuncArgs(fact):
                    f_func, f_arguments, f_negation = getFuncArgs(fact)
                    if f_func==p_func and len(p_arguments)==len(f_arguments):
                        print(f'Fact Function match \'{fact}\'')
                        if unifyable(f_terms=f_arguments, p_terms=p_arguments):
                            print(f'Unifyable with {predicate}')
                            swap = unified_arguments(f_terms=f_arguments, p_terms=p_arguments)
                            unified = predicate
                            for var in swap:
                                unified = replace_var(var=var, rpl=swap[var], sentence=unified)
                            mod = "" if f_negation==p_negation else "not "
                            print(f'After unification {mod}{unified}')
                            cases[f_negation==p_negation].add(unified)
                        else:
                            print(f'Not unifyable with {predicate}')

            # If no Fact is helpfull then maybe a rule inference could be made
            for rule in self.rules:
                if isRule(rule): # A rule is like Abc(x, y) => Pqr(y)
                    condition, implication = getCondImpl(rule) # Abc(x, y), Pqr(y)

                    # If an implication matchs the predicate, then check if condition is true

                    if isFuncArgs(implication):
                        i_func, i_arguments, i_negation = getFuncArgs(implication)
                        if i_func==p_func and len(i_arguments)==len(p_arguments):
                            print(f'Rule match \'{rule}\' for \'{predicate}\'')
                            
                            if unifyable(f_terms=i_arguments, p_terms=p_arguments): # Implication matched
                                print(f'{implication} unifyable with {predicate}')
                                swap = unified_arguments(f_terms=i_arguments, p_terms=p_arguments)
                                unified = predicate
                                for var in swap:
                                    unified = replace_var(var=var, rpl=swap[var], sentence=unified)
                                    rule = replace_var(var=var, rpl=swap[var], sentence=rule)
                                print(f'After unification {rule}')
                                condition, implication = getCondImpl(rule)
                                sub_cases = self.getCases(condition) # Breaks if condition is conjunction
                                if condition in sub_cases[True]: # Condition matched
                                    _, _, i_negation = getFuncArgs(implication)
                                    cases[i_negation==p_negation].add(unified)
                                print(sub_cases)

            return cases 
        return None





    def isTrue(self, predicate: str):
        """This method takes in a predicate and tries to figure out if the predicate is true.
        
        Can not work when inference requires making a guess about a ground term to see if the 
        predicate is true like in test 8.1. 

        I'm developing method getTrueCases to give the engine the ability to guess (Seen in test 8.0)
        """

        # If the predicate is a conjunction, then split in and check if the individual parts are true
        if conjunction(sentence=predicate):
            conjuncts = getConjuncts(sentence=predicate)
            print(f"Conjunct of {conjuncts}")
            result = True
            for conjunct in conjuncts:
                val = self.isTrue(conjunct)
                if val == False: # val may be False or None
                    return False
                elif val is None:
                    result = None
            return result # A single false makes the whole false, a single None makes the whole None if no false, else its true
        

        # The predicate is like ~Abc(x, y)
        elif isFuncArgs(predicate):
            p_func, p_arguments, p_negation = getFuncArgs(predicate) # Abc, [x, y], ~

            for arg in p_arguments:
                if groundTerm(arg):
                    print(f'Predicate {predicate} has groundterm {arg}')
                    #return

            print(f"Checking {predicate}")

            # First look to see if we know a usefull fact
            for fact in self.facts:
                if isFuncArgs(fact):
                    f_func, f_arguments, f_negation = getFuncArgs(fact)
                    if f_func==p_func and len(p_arguments)==len(f_arguments):
                        print(f'Fact Function match \'{p_func}\'')
                        print(f'p_arguments {p_arguments}, f_arguments {f_arguments}')

                        if swapable(f_terms=f_arguments, p_terms=p_arguments):
                            mod = "" if p_negation == f_negation else "not "
                            print(f'Fact \'{fact}\' |=> {mod}{predicate}')
                            return p_negation == f_negation
                        else:
                            print(f'Fact \'{fact}\' ?=> {predicate}')

            # If no Fact is helpfull then maybe a rule inference could be made
            for rule in self.rules:
                if isRule(rule): # A rule is like Abc(x, y) => Pqr(y)
                    condition, implication = getCondImpl(rule) # Abc(x, y), Pqr(y)

                    # If an implication matchs the predicate, then check if condition is true
                    
                    if isFuncArgs(implication):  
                        i_func, i_arguments, i_negation = getFuncArgs(implication)
                        if i_func==p_func and len(i_arguments)==len(p_arguments):
                            print(f'Rule match \'{rule}\' for \'{predicate}\'')

                            if swapable(f_terms=i_arguments, p_terms=p_arguments): # Implication matched
                                swap = swap_list(f_terms=i_arguments, p_terms=p_arguments)
                                print(f'Swaping {swap}')

                                for var in swap:
                                    condition = replace_var(var=var, rpl=swap[var], sentence=condition)
                                    rule = replace_var(var=var, rpl=swap[var], sentence=rule)
                                
                                print(f'Swapped condition \'{condition}\'')
                                result = self.isTrue(condition) 
                                if result == True: # Condition matched
                                    print(f'Rule {rule}')
                                    return result if p_negation == i_negation else not result


        return None # I know I dont need to specify this in python, but its helpfull to see where the unexpected None came from

# Section 3: Tests

I think the easiest way to see what has been done is to start from the earliest tests at the bottom and read upwards.  

## Test 8

These tests are not cleared. They can be ignored if you are simply checking what has been done.

If predicate has ground terms, then is doesn't make sense to ask is_true(predicate)

Instead ask for a list of true predicates with ground terms filled in.

In [73]:
print("Test 8.2.1") # Not passed
def test():
    facts = {
        "TeamAvatarMember(Sokka)",
        "Scarcastic(Sokka)",
        "Prodigy(Azula)",
        "Villan(Azula)",
        "Lethal(Azula)",
    }
    rules = {
        "TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)",
        "Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)", # Prodigy(y) swap y will swap y in both name and argument
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.trueCase("Funny(Sokka) ^ Scary(Azula)"))

test()

Test 8.2.1
Conjunct of {'Funny(Sokka)', 'Scary(Azula)'}
Getting cases for Funny(Sokka)
Rule match 'TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)' for 'Funny(Sokka)'
Funny(x) unifyable with Funny(Sokka)
After unification TeamAvatarMember(Sokka) ^ Scarcastic(Sokka) => Funny(Sokka)


TypeError: ignored

In [74]:
print("Test 8.0.3") # Passed

print("\n\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Smart(Katara)", "Funny(Sokka)", "~Funny(Katara)"})
print( engine_1.trueCase("Funny(Sokka)") )

print("\n\nTest 2")
print( engine_1.trueCase("~Funny(Sokka)") )

print("\n\nTest 3")
print( engine_1.trueCase("Funny(Katara)") )

print("\n\nTest 4")
print( engine_1.trueCase("Funny(Sokka) ^ ~Funny(Katara)") )

Test 8.0.3


Test 1
Getting cases for Funny(Sokka)
Fact Function match 'Funny(Sokka)'
Unifyable with Funny(Sokka)
After unification Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with Funny(Sokka)
True


Test 2
Getting cases for ~Funny(Sokka)
Fact Function match 'Funny(Sokka)'
Unifyable with ~Funny(Sokka)
After unification not ~Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with ~Funny(Sokka)
False


Test 3
Getting cases for Funny(Katara)
Fact Function match 'Funny(Sokka)'
Not unifyable with Funny(Katara)
Fact Function match '~Funny(Katara)'
Unifyable with Funny(Katara)
After unification not Funny(Katara)
False


Test 4
Conjunct of {'Funny(Sokka)', '~Funny(Katara)'}
Getting cases for Funny(Sokka)
Fact Function match 'Funny(Sokka)'
Unifyable with Funny(Sokka)
After unification Funny(Sokka)
Fact Function match '~Funny(Katara)'
Not unifyable with Funny(Sokka)
Getting cases for ~Funny(Katara)
Fact Function match 'Funny(Sokka)'
Not unifyable with ~Funny(Kat

In [39]:
print("Test 8.0.2") # Passed

print("\n\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Smart(Katara)", "Funny(Sokka)"})
print( engine_1.getCases("Funny(x)") )

print("\n\nTest 2")
print( engine_1.getCases("Smart(x)") )

print("\n\nTest 3")
print( engine_1.getCases("Smart(x) ^ Funny(x)") )

Test 8.0.2


Test 1
Getting cases for Funny(x)
Fact Function match 'Funny(Sokka)'
Unifyable with Funny(x)
After unification Funny(Sokka)
{True: {'Funny(Sokka)'}, False: set()}


Test 2
Getting cases for Smart(x)
Fact Function match 'Smart(Sokka)'
Unifyable with Smart(x)
After unification Smart(Sokka)
Fact Function match 'Smart(Katara)'
Unifyable with Smart(x)
After unification Smart(Katara)
{True: {'Smart(Sokka)', 'Smart(Katara)'}, False: set()}


Test 3
None


In [40]:
print("Test 8.0.1") # Passes

print("Test 8.0.1.A")
engine_1 = Fol_engine(facts={"Hostile(FireNation)"}, rules={"Hostile(x) => Enemy(x)"})
print( engine_1.getCases("Enemy(FireNation)") )

print("\n\nTest 8.0.1.B")
engine_1 = Fol_engine(facts={"Hostile(FireNation)"}, rules={"Hostile(x) => Enemy(x)"})
print( engine_1.getCases("~Enemy(FireNation)") )

print("\n\nTest 8.0.1.C")
engine_1 = Fol_engine(facts={"Friendly(AirNomads)"}, rules={"Friendly(x) => ~Enemy(x)"})
print( engine_1.getCases("Enemy(AirNomads)") )

Test 8.0.1
Test 8.0.1.A
Getting cases for Enemy(FireNation)
Rule match 'Hostile(x) => Enemy(x)' for 'Enemy(FireNation)'
Enemy(x) unifyable with Enemy(FireNation)
After unification Hostile(FireNation) => Enemy(FireNation)
Getting cases for Hostile(FireNation)
Fact Function match 'Hostile(FireNation)'
Unifyable with Hostile(FireNation)
After unification Hostile(FireNation)
{True: {'Hostile(FireNation)'}, False: set()}
{True: {'Enemy(FireNation)'}, False: set()}


Test 8.0.1.B
Getting cases for ~Enemy(FireNation)
Rule match 'Hostile(x) => Enemy(x)' for '~Enemy(FireNation)'
Enemy(x) unifyable with ~Enemy(FireNation)
After unification Hostile(FireNation) => Enemy(FireNation)
Getting cases for Hostile(FireNation)
Fact Function match 'Hostile(FireNation)'
Unifyable with Hostile(FireNation)
After unification Hostile(FireNation)
{True: {'Hostile(FireNation)'}, False: set()}
{True: set(), False: {'~Enemy(FireNation)'}}


Test 8.0.1.C
Getting cases for Enemy(AirNomads)
Rule match 'Friendly(x) => 

In [41]:
print("Test 8.0") # Passed

facts={
    "TeamAvatarMember(Aang)", 
    "TeamAvatarMember(Sokka)", 
    "TeamAvatarMember(Katara)", 
    "TeamAvatarMember(Toph)", 
    "TeamAvatarMember(Suki)",
    "~TeamAvatarMember(Bumi)",
    "~TeamAvatarMember(Iroh)"
}

engine_1 = Fol_engine(facts=facts)
engine_1.isTrue("TeamAvatarMember(x)")
print( engine_1.getCases("TeamAvatarMember(x)") )

print("\n\nTest 8.0.B")
print( engine_1.getCases("~TeamAvatarMember(x)") )

print("\n\nTest 8.0.C")
print( engine_1.getCases("TeamAvatarMember(Toph)") )

print("\n\nTest 8.0.D")
print( engine_1.getCases("TeamAvatarMember(Bumi)") )

print("\n\nTest 8.0.E")
print( engine_1.getCases("~TeamAvatarMember(Iroh)") )


Test 8.0
Predicate TeamAvatarMember(x) has groundterm x
Checking TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Toph']
Fact 'TeamAvatarMember(Toph)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Katara']
Fact 'TeamAvatarMember(Katara)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Sokka']
Fact 'TeamAvatarMember(Sokka)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Suki']
Fact 'TeamAvatarMember(Suki)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Bumi']
Fact '~TeamAvatarMember(Bumi)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Aang']
Fact 'TeamAvatarMember(Aang)' ?=> TeamAvatarMember(x)
Fact Function match 'TeamAvatarMember'
p_arguments ['x'], f_arguments ['Iroh']
Fact '~TeamAvatarMember(Iroh)' 

In [ ]:
print("Test 8.1") # Not Passed

"""
Fact 'Hostile(Nono)' ?=> Hostile(z)
Under normal condition asking Hostile(z) would be interpretted as is everyone hostile, since z is a ground term and 
that means the predicate should remain true regardless of what the ground term is replaced with.

"""


def test():
    facts = {
        "American(CoWest)",
        "Sells(CoWest, Missile, Nono)",
        "Hostile(Nono)",
        "Has(Nono, Missile)",
        "Weapon(Missile)",

    }
    rules = {
        "American(x) ^ Weapon(y) ^ Enemy(z) ^ Sold(x, y, z) => Criminal(x)",
        "Hostile(w) => Enemy(w)",
        "Sells(p, q, r) ^ Has(r, q) => Sold(p, q, r)"
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.isTrue("Criminal(CoWest)"))

test()

Test 8.1
Checking Criminal(CoWest)
Rule match 'American(x) ^ Weapon(y) ^ Enemy(z) ^ Sold(x, y, z) => Criminal(x)' for 'Criminal(CoWest)'
Swaping {'x': 'CoWest'}
Swapped condition 'American(CoWest) ^ Weapon(y) ^ Enemy(z) ^ Sold(CoWest, y, z)'
Conjunct of {'Enemy(z)', 'Weapon(y)', 'Sold(CoWest, y, z)', 'American(CoWest)'}
Predicate Enemy(z) has groundterm z
Checking Enemy(z)
Rule match 'Hostile(w) => Enemy(w)' for 'Enemy(z)'
Swaping {'w': 'z'}
Swapped condition 'Hostile(z)'
Predicate Hostile(z) has groundterm z
Checking Hostile(z)
Fact Function match 'Hostile'
p_arguments ['z'], f_arguments ['Nono']
Fact 'Hostile(Nono)' ?=> Hostile(z)
Predicate Weapon(y) has groundterm y
Checking Weapon(y)
Fact Function match 'Weapon'
p_arguments ['y'], f_arguments ['Missile']
Fact 'Weapon(Missile)' ?=> Weapon(y)
Predicate Sold(CoWest, y, z) has groundterm y
Predicate Sold(CoWest, y, z) has groundterm z
Checking Sold(CoWest, y, z)
Rule match 'Sells(p, q, r) ^ Has(r, q) => Sold(p, q, r)' for 'Sold(CoWest,

## Test 7

These tests (7 and below) have been passed and they demonstrate the current capabalities of my current FOL engine.

In [ ]:
print("Test 7.1")
def test():
    facts = {
        "TeamAvatarMember(Sokka)",
        "Scarcastic(Sokka)",
        "Prodigy(Azula)",
        "Villan(Azula)",
        "Lethal(Azula)",
    }
    rules = {
        "TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)",
        "Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)", # Prodigy(y) swap y will swap y in both name and argument
    }
    engine_1 = Fol_engine(facts=facts, rules=rules)
    print(engine_1.isTrue("Funny(Sokka) ^ Scary(Azula)"))

test()

Test 7.1
Conjunct of {'Scary(Azula)', 'Funny(Sokka)'}
Checking Scary(Azula)
Rule match 'Villan(y) ^ Lethal(y) ^ Prodigy(y) => Scary(y)' for 'Scary(Azula)'
Swaping {'y': 'Azula'}
Swapped condition 'Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula)'
Conjunct of {'Villan(Azula)', 'Lethal(Azula)', 'Prodigy(Azula)'}
Checking Villan(Azula)
Fact Function match 'Villan'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Villan(Azula)' |=> Villan(Azula)
Checking Lethal(Azula)
Fact Function match 'Lethal'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Lethal(Azula)' |=> Lethal(Azula)
Checking Prodigy(Azula)
Fact Function match 'Prodigy'
p_arguments ['Azula'], f_arguments ['Azula']
Fact 'Prodigy(Azula)' |=> Prodigy(Azula)
Rule Villan(Azula) ^ Lethal(Azula) ^ Prodigy(Azula) => Scary(Azula)
Checking Funny(Sokka)
Rule match 'TeamAvatarMember(x) ^ Scarcastic(x) => Funny(x)' for 'Funny(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'TeamAvatarMember(Sokka) ^ Scarcastic(Sokka)'
Conjunct of {'TeamAva

## Tests 6 and below

In [34]:
# Test 6
# and now the engine understand 'and'
print("\nTest 1")
engine_1 = Fol_engine(facts={"Smart(Sokka)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
engine_1.isTrue("~Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 4")
engine_1 = Fol_engine(facts={"Smart(x)", "~Funny(Sokka)"})
engine_1.isTrue("Smart(Sokka) ^ Funny(Sokka)")

print("\nTest 5")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))

print("\nTest 6")
engine_1 = Fol_engine(facts={"Smart(x)", "Funny(Sokka)", "~Funny(Azula)"})
print(engine_1.isTrue("Smart(Sokka) ^ Funny(Azula)"))


Test 1
Conjunct of {'Funny(Sokka)', 'Smart(Sokka)'}
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Smart(Sokka)' |=> Smart(Sokka)

Test 2
Conjunct of {'Funny(Sokka)', 'Smart(Sokka)'}
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)

Test 3
Conjunct of {'Funny(Sokka)', '~Smart(Sokka)'}
Checking Funny(Sokka)
Fact Function match 'Funny'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Funny(Sokka)' |=> Funny(Sokka)
Checking ~Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> not ~Smart(Sokka)

Test 4
Conjunct of {'Funny(Sokka)', 'Smart(Sokka)'}
Check

In [33]:
# Test 5
# Bringing in rules so that the engine can do some implication logic

print("\nTest 1")
engine_1 = Fol_engine(facts={"Clever(Sokka)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Sokka)") )


print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Sokka)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(y)"}, rules={"Clever(x) => ~Silly(x)"})
print( engine_1.isTrue("Silly(Sokka)") )

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(Sokka)"}, rules={"Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Sokka)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"Clever(Sokka)"}, rules={"~Clever(x) => Smart(x)"})
print( engine_1.isTrue("Smart(Sokka)") )


Test 1
Checking Smart(Sokka)
Rule match 'Clever(x) => Smart(x)' for 'Smart(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'Clever(Sokka)'
Checking Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Clever(Sokka)' |=> Clever(Sokka)
Rule Clever(Sokka) => Smart(Sokka)
True

Test 2
Checking Smart(Sokka)
Rule match 'Clever(x) => Smart(x)' for 'Smart(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'Clever(Sokka)'
Checking Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Sokka)
Rule Clever(Sokka) => Smart(Sokka)
True

Test 3
Checking Silly(Sokka)
Rule match 'Clever(x) => ~Silly(x)' for 'Silly(Sokka)'
Swaping {'x': 'Sokka'}
Swapped condition 'Clever(Sokka)'
Checking Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['y']
Fact 'Clever(y)' |=> Clever(Sokka)
Rule Clever(Sokka) => ~Silly(Sokka)
False

Test 4
Checking Smart(Sokka)
Rule match 'Clever(x) => Smart(x

In [32]:
# Test 4
# I've added negation into the mix

print("\nTest 1")
engine_1 = Fol_engine(facts={"~Clever(Sokka)"})
print( engine_1.isTrue("Clever(Sokka)") ) # False because it is known that Sokka is not Clever

print("\nTest 2")
engine_1 = Fol_engine(facts={"~Clever(Sokka)"})
print( engine_1.isTrue("~Clever(Sokka)") )

print("\nTest 3")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("Clever(Sokka)") ) # False because it is known that everyone is not clever

print("\nTest 4")
engine_1 = Fol_engine(facts={"~Clever(x)"})
print( engine_1.isTrue("~Clever(Sokka)") )

print("\nTest 5")
engine_1 = Fol_engine(facts={"~Clever(Sokka)"})
print( engine_1.isTrue("Clever(x)") ) # No idea about everyone, engine just knows about Sokka
"""                                  Future Rahul here:
                                        This way of using ground terms kinda comes back to bite me
                                        I dont have a way to ask for what cases would make this predicate 
                                        true or false. 
"""

print("\nTest 6")
engine_1 = Fol_engine(facts={"~Clever(Sokka)"})
print( engine_1.isTrue("~Clever(x)") )


Test 1
Checking Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact '~Clever(Sokka)' |=> not Clever(Sokka)
False

Test 2
Checking ~Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact '~Clever(Sokka)' |=> ~Clever(Sokka)
True

Test 3
Checking Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['x']
Fact '~Clever(x)' |=> not Clever(Sokka)
False

Test 4
Checking ~Clever(Sokka)
Fact Function match 'Clever'
p_arguments ['Sokka'], f_arguments ['x']
Fact '~Clever(x)' |=> ~Clever(Sokka)
True

Test 5
Predicate Clever(x) has groundterm x
Checking Clever(x)
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Sokka']
Fact '~Clever(Sokka)' ?=> Clever(x)
None

Test 6
Predicate ~Clever(x) has groundterm x
Checking ~Clever(x)
Fact Function match 'Clever'
p_arguments ['x'], f_arguments ['Sokka']
Fact '~Clever(Sokka)' ?=> ~Clever(x)
None


In [42]:
 # Test 3
# Just checking if the the engine still works with multi argument predicates

print('Test 1')
engine_1 = Fol_engine(facts={"Has(x, Pen)"})
engine_1.isTrue("Has(Sokka, Pen)")

print('Test 2')
engine_1 = Fol_engine(facts={"OwnedBy(Pen, x)"})
engine_1.isTrue("OwnedBy(Pen, Sokka)") 

Test 1
Checking Has(Sokka, Pen)
Fact Function match 'Has'
p_arguments ['Sokka', 'Pen'], f_arguments ['x', 'Pen']
Fact 'Has(x, Pen)' |=> Has(Sokka, Pen)
Test 2
Checking OwnedBy(Pen, Sokka)
Fact Function match 'OwnedBy'
p_arguments ['Pen', 'Sokka'], f_arguments ['Pen', 'x']
Fact 'OwnedBy(Pen, x)' |=> OwnedBy(Pen, Sokka)


True

In [43]:
 # Test 2
'''
Engine should unify "Smart(Rahul)" with "Smart(x)"
first recognize "Smart(x)" is FuncArg
then recognize argument x is ground term, so itmight be unified with something
try to unify:
    success: is this the result?
    fail: move on
'''

print('Test 1')
engine_1 = Fol_engine(facts={"Smart(x)"})  # for all x, Smart(x) is true
engine_1.isTrue("Smart(Sokka)")

print("\nTest 2")
engine_1 = Fol_engine(facts={"Clever(x)", "Smart(y)"})
engine_1.isTrue("Smart(Sokka)")

print("\nTest 3")
engine_1 = Fol_engine(facts={"Clever(x)"}) # Only known fact is that everyone is Clever
engine_1.isTrue("Smart(Sokka)") == None # No idea about being Smart

Test 1
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['x']
Fact 'Smart(x)' |=> Smart(Sokka)

Test 2
Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['y']
Fact 'Smart(y)' |=> Smart(Sokka)

Test 3
Checking Smart(Sokka)


True

In [30]:
# Test 1
engine_1 = Fol_engine(facts={"Smart(Sokka)"}) # Known Fact Sokka is smart
engine_1.isTrue("Smart(Sokka)") # Engine can confidently say that Sokka is indeed smart

Checking Smart(Sokka)
Fact Function match 'Smart'
p_arguments ['Sokka'], f_arguments ['Sokka']
Fact 'Smart(Sokka)' |=> Smart(Sokka)


True